Perform imports and fetch the page source

In [80]:
import pandas as pd # library for data analsysis
import requests

from lxml import html
from lxml import etree

page_link = 'http://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&rvsection=0&titles=List_of_postal_codes_of_Canada:_M'
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

page_src = response.text
print('done')


done


Use an xpath to find the table in the page source.  Each row of the table is an element in tlist. 

Loop through each element in the list, split it to get the individual table cells.  Those cells can then be cleaned up and added to a dataframe.

If we see the same postcode twice, combine the rows.  

In [144]:
el = etree.fromstring(page_src)
tlist = el.xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr')
tlist = tlist[1:]

column_names = ['Postcode', 'Borough', 'Neighborhood'] 
df = pd.DataFrame(columns=column_names)

count = 0
for elem in tlist:
    row_src = etree.tostring(elem, pretty_print=True, method="text").decode("utf-8")
    row = row_src.split('\n')

    if (row[2] == 'Not assigned'): continue    
    if (row[3] == 'Not assigned'): row[3] = row[2]
        
    postcode = row[1]
    
    if (postcode == last_postcode):
        val = df.loc[count-1, 'Neighborhood']
        df.loc[count-1, 'Neighborhood'] = val + ', ' + row[3]
    else:
        df = df.append({'Postcode': postcode,
                    'Borough': row[2],
                    'Neighborhood': row[3],
                    }, ignore_index=True)
        count = count + 1
    last_postcode = postcode
    
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Show the final shape of the table.

In [146]:
df.shape

(103, 3)